<a href="https://colab.research.google.com/github/CidClayQuirino/ConditionMonitoring_RNN/blob/main/DeslocamentoBrazauro02Set2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install markdown
!pip install statsmodels
!pip install scikit-learn
!pip install PyGithub
!pip install gitpython
!pip install statsmodels
!pip install dash
!pip install xlwt
!pip install openpyxl
!pip install tensorflow
!pip install scipy

In [8]:
import pandas as pd
import requests
from io import BytesIO
import plotly.express as px

# Definir o URL base do repositório
url_base = "https://raw.githubusercontent.com/CidClayQuirino/ConditionMonitoring_RNN/main/"

# Definir os nomes dos arquivos e os nomes dos equipamentos correspondentes
arquivos = [
    ("050-003 2023 2024.xlsx", "050-003"),
    ("050-002 2023 2024.xlsx", "050-002"),
    ("050-001 2023 2024.xlsx", "050-001")
]

# Definir as colunas que queremos manter
colunas_desejadas = [
    "Date",
    "Distance Traveled (mi)"
]

# Função para baixar e ler o arquivo
def baixar_e_ler_excel(url):
    resposta = requests.get(url)
    resposta.raise_for_status()  # Para garantir que o download foi bem-sucedido
    arquivo = BytesIO(resposta.content)
    return pd.read_excel(arquivo, usecols=colunas_desejadas)

# Ler os arquivos, adicionar a coluna de nome do equipamento, e manter as colunas desejadas
dfs = []
for arquivo, equipamento in arquivos:
    url_arquivo = url_base + arquivo
    df = baixar_e_ler_excel(url_arquivo)
    # Adicionar a coluna "Equipment Name"
    df['Equipment Name'] = equipamento
    # Ordenar o DataFrame pela data mais antiga para a mais recente
    df = df.sort_values(by='Date')
    # Converter de milhas para quilômetros
    df['Distance Traveled (km)'] = df['Distance Traveled (mi)'] * 1.60934
    # Criar a coluna acumulada para "Distance Traveled (km)"
    df['Cumulative Distance Traveled (km)'] = df['Distance Traveled (km)'].cumsum()
    dfs.append(df)

# Concatenar os DataFrames em um único DataFrame
df_final = pd.concat(dfs, ignore_index=True)

# Exibir as primeiras linhas para verificar
print(df_final.head())

# Gerar o gráfico interativo
fig = px.line(
    df_final,
    x='Date',
    y='Cumulative Distance Traveled (km)',
    color='Equipment Name',  # Diferenciar por equipamento
    title='Cumulative Distance Traveled Over Time (KM) by Equipment',
    labels={'Date': 'Date', 'Cumulative Distance Traveled (km)': 'Cumulative Distance Traveled (KM)'}
)

# Adicionar interatividade
fig.update_traces(mode='lines+markers')
fig.update_layout(hovermode='x unified')

# Mostrar o gráfico interativo
fig.show()


/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning:

Workbook contains no default style, apply openpyxl's default

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning:

Workbook contains no default style, apply openpyxl's default

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning:

Workbook contains no default style, apply openpyxl's default



         Date  Distance Traveled (mi) Equipment Name  Distance Traveled (km)  \
0  2023-01-01                     NaN        050-003                     NaN   
1  2023-01-02                     NaN        050-003                     NaN   
2  2023-01-03                     NaN        050-003                     NaN   
3  2023-01-04                     NaN        050-003                     NaN   
4  2023-01-05                     NaN        050-003                     NaN   

   Cumulative Distance Traveled (km)  
0                                NaN  
1                                NaN  
2                                NaN  
3                                NaN  
4                                NaN  
